## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716830403805, experiment_id='0', last_update_time=1716830403805, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/27 19:20:15 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'mlflow-artifacts:/1/c8ba08a44cdd49bab2c2a014f43d47f2/artifacts'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1716830415289, experiment_id='1', last_update_time=1716830415289, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716830403805, experiment_id='0', last_update_time=1716830403805, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [7]:
client.search_registered_models()

[]

In [21]:
mlflow.search_runs()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.C,params.random_state,tags.mlflow.source.name,tags.mlflow.log-model.history,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.runName
0,c8ba08a44cdd49bab2c2a014f43d47f2,1,FINISHED,mlflow-artifacts:/1/c8ba08a44cdd49bab2c2a014f4...,2024-05-27 17:20:15.523000+00:00,2024-05-27 17:20:22.037000+00:00,0.96,0.1,42,e:\Documentos\Proyectos\Personal\mlops-zoomcam...,"[{""run_id"": ""c8ba08a44cdd49bab2c2a014f43d47f2""...",LOCAL,Almudena,smiling-gnat-844


In [18]:
run_id = client.search_runs(experiment_ids=['1'])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/05/27 19:24:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716830647326, current_stage='None', description='', last_updated_timestamp=1716830647326, name='iris-classifier', run_id='c8ba08a44cdd49bab2c2a014f43d47f2', run_link='', source='mlflow-artifacts:/1/c8ba08a44cdd49bab2c2a014f43d47f2/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [19]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1716830647288, description='', last_updated_timestamp=1716830647326, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716830647326, current_stage='None', description='', last_updated_timestamp=1716830647326, name='iris-classifier', run_id='c8ba08a44cdd49bab2c2a014f43d47f2', run_link='', source='mlflow-artifacts:/1/c8ba08a44cdd49bab2c2a014f43d47f2/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]